<a href="https://colab.research.google.com/github/ykitaguchi77/ErrorDiffusion/blob/main/MNIST_ErrorDiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Error Diffusion MNIST**

https://qiita.com/pocokhc/items/f7ab56051bb936740b8f

In [ ]:

import math
import random

random.seed(10)  # 乱数のシードを設定


def sign(x):  # 符号関数の定義
    if x == 0:
        return 0
    return 1 if x > 0 else -1


def relu(x):  # ReLU関数の定義
    return x if x > 0 else 0


def relu_derivative(x):  # ReLU関数の導関数の定義
    return 1 if x > 0 else 0


def sigmoid(x, u0=0.4):  # シグモイド関数の定義
    return 1 / (1 + math.exp(-2 * x / u0))


def sigmoid_derivative(x):  # シグモイド関数の導関数の定義
    return sigmoid(x) * (1 - sigmoid(x))


def linear(x):  # 線形関数の定義
    return x


def linear_derivative(x):  # 線形関数の導関数の定義
    return 1


class Neuron:  # ニューロンクラスの定義
    def __init__(
        self,
        in_neurons: list["Neuron"],  # 入力ニューロンのリスト
        ntype: str,  # ニューロンのタイプ（"p": positive, "n": negative）
        alpha: float = 0.8,  # 学習率
        activation=sigmoid,  # 活性化関数
        activation_derivative=sigmoid_derivative,  # 活性化関数の導関数
    ) -> None:
        self.ntype = ntype
        self.alpha = alpha
        self.activation = activation
        self.activation_derivative = activation_derivative

        # --- 重みの初期化
        self.weights = []
        for n in in_neurons:
            if ntype == "p":
                if n.ntype == "p":
                    ope = 1
                else:
                    ope = -1
            else:
                if n.ntype == "p":
                    ope = -1
                else:
                    ope = 1
            self.weights.append(random.random() * ope)

        # --- 演算子の設定
        self.operator = 1 if ntype == "p" else -1
        self.weights_operator = [n.operator for n in in_neurons]

        # --- 更新インデックスの設定
        self.upper_idx_list = []
        self.lower_idx_list = []
        for i, n in enumerate(in_neurons):
            if n.ntype == "p":
                self.upper_idx_list.append(i)
            else:
                self.lower_idx_list.append(i)

    def forward(self, x):  # 順伝播の計算
        assert len(self.weights) == len(x)
        y = [x[i] * self.weights[i] for i in range(len(self.weights))]
        y = sum(y)
        self.prev_in = x
        self.prev_out = y
        y = self.activation(y)
        return y

    def update_weight(self, delta_out, direct: str):  # 重みの更新
        grad = self.activation_derivative(abs(self.prev_out))

        if direct == "upper":
            indices = self.upper_idx_list
        else:
            indices = self.lower_idx_list

        for idx in indices:
            _old_w = self.weights[idx]
            delta = self.alpha * self.prev_in[idx]
            delta *= grad
            delta *= delta_out * self.operator * self.weights_operator[idx]
            self.weights[idx] += delta

            # --- デバッグ用の出力
            s = f"{idx:2d}"
            s += f", ot_in {self.prev_in[idx]:5.2f}"
            s += f", f'({self.prev_out:5.2f})={grad:5.2f}"
            s += f", del_ot {delta_out:5.2f}"
            s += f", d {delta:6.3f}"
            s += f", {self.operator:2d} {self.weights_operator[idx]:2d}"
            s += f", w {_old_w:5.2f} -> {self.weights[idx]:5.2f}"
            # print(s)

    def __str__(self):  # ニューロンの情報を文字列で返す
        s = f"{self.ntype} {self.operator:2d}"
        arr = []
        for i in range(len(self.weights)):
            o = "+" if i in self.upper_idx_list else "-"
            arr.append(f"{self.weights[i]:6.3f}({self.weights_operator[i]:2d},{o})")
        s += " [" + ", ".join(arr) + "]"
        return s


class ThreeLayerModel:  # 3層ニューラルネットワークモデルの定義
    def __init__(
        self,
        input_num: int,  # 入力層のニューロン数
        hidden_num: int,  # 中間層のニューロン数
        alpha: float = 0.8,  # 学習率
        beta: float = 0.8,  # バイアス
    ) -> None:
        self.beta = beta

        # 中間層のバイアスニューロン
        hd_p = Neuron([], "p")
        hd_n = Neuron([], "n")

        # 入力層のニューロン
        inputs: list[Neuron] = []
        for i in range(input_num):
            inputs.append(Neuron([], "p"))
            inputs.append(Neuron([], "n"))

        # 中間層のニューロン
        self.hidden_neurons: list[Neuron] = []
        for i in range(hidden_num):
            self.hidden_neurons.append(
                Neuron(
                    [hd_p, hd_n] + inputs,
                    ntype=("p" if i % 2 == 1 else "n"),
                    alpha=alpha,
                    activation=sigmoid,
                    activation_derivative=sigmoid_derivative,
                )
            )

        # 出力層のニューロン
        self.out_neuron = Neuron(
            [hd_p, hd_n] + self.hidden_neurons,
            "p",
            alpha=alpha,
            activation=sigmoid,
            activation_derivative=sigmoid_derivative,
        )

    def forward(self, inputs):  # 順伝播の計算
        # 入力層
        x = []
        for n in inputs:
            x.append(n)  # p
            x.append(n)  # n

        # 中間層
        x = [h.forward([self.beta, self.beta] + x) for h in self.hidden_neurons]

        # 出力層
        x = self.out_neuron.forward([self.beta, self.beta] + x)

        return x

    def train(self, inputs, target):  # 学習
        x = self.forward(inputs)

        # --- 重みの更新（ED法）
        diff = target - x
        if diff > 0:
            direct = "upper"
        else:
            direct = "lower"
            diff = -diff
        self.out_neuron.update_weight(diff, direct)
        for h in self.hidden_neurons:
            h.update_weight(diff, direct)

        return diff


def main_one():  # 恒等関数の学習
    model = ThreeLayerModel(1, hidden_num=1)

    # --- 学習ループ
    dataset = [
        [0, 1],
        [1, 1],
    ]
    for i in range(100):
        x, target = dataset[random.randint(0, len(dataset)) - 1]
        metric = model.train([x], target)

        # --- 予測
        y = model.forward([x])
        print(f"{i} in{x:5.2f} -> {y:5.2f}, target {target:5.2f}, metric {metric:5.2f}")

    print("--- result ---")
    for x, target in dataset:
        y = model.forward([x])
        print(f"{x:5.2f} -> {y:5.2f}, target {target:5.2f}")

    # --- 最終的な重み
    print("--- weights ---")
    print(model.out_neuron)
    for n in model.hidden_neurons:
        print(n)


def main_not():  # NOT関数の学習
    model = ThreeLayerModel(1, hidden_num=2)

    # --- 学習ループ
    dataset = [
        [0, 1],
        [1, 0],
    ]
    for i in range(100):
        x, target = dataset[random.randint(0, len(dataset)) - 1]
        metric = model.train([x], target)

        # --- 予測
        y = model.forward([x])
        print(f"{i} in{x:5.2f} -> {y:5.2f}, target {target:5.2f}, metric {metric:5.2f}")

    print("--- result ---")
    for x, target in dataset:
        y = model.forward([x])
        print(f"{x:5.2f} -> {y:5.2f}, target {target:5.2f}")

    # --- 最終的な重み
    print("--- weights ---")
    print(model.out_neuron)
    for n in model.hidden_neurons:
        print(n)


def main_xor():  # XOR関数の学習
    model = ThreeLayerModel(2, hidden_num=16)

    # --- 学習ループ
    dataset = [
        [0, 0, 1.0],
        [1, 0, 0.0],
        [0, 1, 0.0],
        [1, 1, 1.0],
    ]
    for i in range(100):
        x1, x2, target = dataset[random.randint(0, len(dataset)) - 1]
        metric = model.train([x1, x2], target)

        # --- 予測
        y = model.forward([x1, x2])
        print(f"{i} in[{x1:5.2f},{x2:5.2f}] -> {y:5.2f}, target {target:5.2f}, metric {metric:5.2f}")

    print("--- result ---")
    for x1, x2, target in dataset:
        y = model.forward([x1, x2])
        print(f"[{x1:5.2f},{x2:5.2f}] -> {y:5.2f}, target {target:5.2f}")

    # --- 最終的な重み
    print("--- weights ---")
    print(model.out_neuron)
    for n in model.hidden_neurons:
        print(n)


if __name__ == "__main__":
    main_one()  # 恒等関数の学習を実行
    main_not()  # NOT関数の学習を実行
    main_xor()  # XOR関数の学習を実行
```

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(28 * 28)),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(16, activation="sigmoid"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)


if __name__ == "__main__":
    main_one()
    main_not()
    main_xor()